In [1]:
import yfinance
import stock_utils
import pandas as pd
import numpy as np

from bar import Bar

In [2]:
# fetching liquit tickers, vol > 4M
tickers = stock_utils.fetch_liquid_stocks(vol=2_000_000, min_price=2)
print(f"Fetched {len(tickers)} tickers")

Fetched 549 tickers


In [3]:
#ignored tickers
ignored = stock_utils.get_ignored_symbols() or []
','.join(ignored)

'ACTT,AYR,AMTD,GDI,BPR,LPT,VSLR,ARQL,MDCO,MNTA,PEGI,STML,ERI,HABT,WLH,VTIQ,CY,CSFL,LVGO,CTL,APY,LM,UTX,ZAYO,CAR,MYL,NBL,RARX,AGN,GMHI,FSB,MYOK,AMAG,BWMC,ADSW,PKD,FMCI,AOBC,FII,TLRA,SNH,WBC,DERM,WTR,AVX,GRAF,FSCT,BOLD,HIIQ,IOTS,TRNE,EROS,DRAD,PDLI,GCAP,RUBI,CHK,PE,REAL'

In [4]:
filtered_tickers = list(set(tickers) - set(ignored))
print(f"Total {len(filtered_tickers)} filtered tickers")

Total 499 filtered tickers


In [5]:
#Historical data for filtered tickers
data = {}
for t in filtered_tickers:
    try:
        s = yfinance.Ticker(t)
        data[t] =  s.history(period='4d')
        data[t].reset_index(inplace=True)
        data[t].columns = data[t].columns.str.lower()
        data[t] = data[t].iloc[::-1,:].reset_index(drop=True)
    except :
        print(t)

- HDS: No data found for this date range, symbol may be delisted
- RMG: No data found for this date range, symbol may be delisted
- NTGN: No data found, symbol may be delisted


In [6]:
patterns_list2 = []
for key in data.keys():
    if data[key].empty or len(data[key]) < 2:
        continue
    try:
        bar1 = Bar(
            data[key].loc[0, 'open'],
            data[key].loc[0, 'high'],
            data[key].loc[0, 'low'],
            data[key].loc[0, 'close'],
        )
        bar2 = Bar(
            data[key].loc[1, 'open'],
            data[key].loc[1, 'high'],
            data[key].loc[1, 'low'],
            data[key].loc[1, 'close']
        )
        if bar1  <= bar2:
            patterns_list2.append(key)
    except ValueError as e:
        print(e)
    except Exception as e:
        print(e)

','.join(set(patterns_list2))

Open price must be between High and Low:
 High:39.189998626708984 => Open:39.22999954223633 >= Low:34.7599983215332
Open price must be between High and Low:
 High:15.020000457763672 => Open:14.609999656677246 >= Low:14.619999885559082


'DRH,AQMS,SHLX,ED,CLBS,PRVL,SNAP,KDMN,AKBA,CXO,BK,AM,WATT,ADT,MCD,DARE,HD,FOLD,TOL,AVGR,CLVS,WELL,CHNG,IBM,DIS,GEVO,TMHC,GOOG,INVH,TJX,FAST,LEN,VISL,VTR,ATVI,DGLY,IVZ,NOV,VXRT,AMGN,DUK,FOXA,K,DELL,MBIO,ICLK,DVAX,FIS,SAVE,WY,CGIX,MARA,MPW,TMUS,INSG,GS,DT,HBI,KHC,LKQ,NEE,QCOM,ON,ALXN,CDEV,SWN,FCX,NDRA,D,KSS,ETSY,NCLH,PSEC,X,VST,IDEX,CI,WORK,JBLU,GLW,RESN,MSFT,NVAX,JWN,BRX,VICI,LTHM,AVTR,RCL,SBUX,MA,DAL,BCRX,LYFT,CRK,NIO,NWL,KOPN,JPM,CTSH,MIK,COST,LUV,PRTS,SIRI,TSN,FISV,PLD,DHI,AMAT,ADI,CFG,EXEL,WFC,EBAY,AMZN,ADMA,MO,WU,PM,MPC,AMC,VZ,BNGO,SPG,KIM,NEM,COCP,USB,OBLN,APPS,PK,NFLX,ACRS,DRE,WKHS,AAPL,FOX,AIV,CCL,FB,JNPR,INFI,ISBC,AEP,AXTA,ADBE,MAXR,WMT,BYND,TPH,TDOC,AMRS,TELL,COTY,MU,KDP,PINS,SFM,FE,MVIS,FNB,CYRX,EA,GIS,AQB,WYNN,V,MARK,VSTM,STCN,TWTR,BX,LOW,ORGO,RIOT,RRC,HAL,TSLA,CAG,FTI,LITE,GILD,ZM,ATUS,CRM,GNW,SRNE,IPG,COMM,HMHC,LPCN,MMM,CNK,EQR,ROKU,CXW,JCI,CLDR,UBER,SPPI,AR,M,VNO,CLNE,BE,BOXL,VUZI,GRTS,MOS,SEE,KBH,CNX,AHT,PHM,IZEA,SCHW,VLO,KR,NKE,STT,EQH,MS,HL,BLNK,ETRN,EQT'

In [7]:
patterns_list = []
for key in data.keys():
    if data[key].empty or len(data[key]) < 3:
        continue
    try:
        bar1 = Bar(
            data[key].loc[0, 'open'],
            data[key].loc[0, 'high'],
            data[key].loc[0, 'low'],
            data[key].loc[0, 'close'],
        )
        bar2 = Bar(
            data[key].loc[1, 'open'],
            data[key].loc[1, 'high'],
            data[key].loc[1, 'low'],
            data[key].loc[1, 'close'],
        )
        bar3 = Bar(
            data[key].loc[2, 'open'],
            data[key].loc[2, 'high'],
            data[key].loc[2, 'low'],
            data[key].loc[2, 'close'],
        )
        if bar1 <= bar2 <= bar3:
            patterns_list.append(key)
    except ValueError as e:
        print(e)
        
','.join(set(patterns_list))

Open price must be between High and Low:
 High:39.189998626708984 => Open:39.22999954223633 >= Low:34.7599983215332
Open price must be between High and Low:
 High:15.020000457763672 => Open:14.609999656677246 >= Low:14.619999885559082


'AQMS,SNAP,KDMN,DARE,AVGR,CLVS,WELL,CHNG,TJX,GEVO,VISL,ATVI,DGLY,VXRT,DUK,FOXA,K,ICLK,DVAX,TMUS,INSG,GS,HBI,KHC,LKQ,QCOM,CDEV,ETSY,PSEC,IDEX,JWN,AVTR,LYFT,JPM,MIK,COST,SIRI,WFC,EBAY,ADMA,PM,WU,MPC,AMC,BNGO,COCP,ACRS,FOX,CCL,FB,AEP,MAXR,BYND,COTY,MU,SFM,FE,EA,MARK,TWTR,LITE,GILD,ATUS,SRNE,COMM,HMHC,MMM,JCI,CLDR,M,VNO,BE,BOXL,SEE,CNX,IZEA,SCHW,KR,STT,EQT'

In [8]:
patterns_list3 = []
for key in data.keys():
    if data[key].empty or len(data[key]) < 3:
        continue
    try:
        bar1 = Bar(
            data[key].loc[0, 'open'],
            data[key].loc[0, 'high'],
            data[key].loc[0, 'low'],
            data[key].loc[0, 'close'],
        )
        bar2 = Bar(
            data[key].loc[1, 'open'],
            data[key].loc[1, 'high'],
            data[key].loc[1, 'low'],
            data[key].loc[1, 'close'],
        )
        bar3 = Bar(
            data[key].loc[2, 'open'],
            data[key].loc[2, 'high'],
            data[key].loc[2, 'low'],
            data[key].loc[2, 'close'],
        )
        if (bar3.range / bar2.range >= 1.7) and (bar2.range / bar1.range) >= 0.95:
            patterns_list3.append(key)
    except ValueError as e:
        print(e)
        
','.join(patterns_list3)

Open price must be between High and Low:
 High:39.189998626708984 => Open:39.22999954223633 >= Low:34.7599983215332
Open price must be between High and Low:
 High:15.020000457763672 => Open:14.609999656677246 >= Low:14.619999885559082


'WELL,GEVO,VXRT,ICLK,HBI,STT,GILD,INSG,PSEC,ATUS'

# HOMEWORK STOCKS

In [9]:
','.join(set(patterns_list+patterns_list2+patterns_list3))

'DRH,AQMS,SHLX,ED,CLBS,PRVL,SNAP,KDMN,AKBA,CXO,BK,AM,WATT,DARE,ADT,MCD,HD,FOLD,TOL,AVGR,CLVS,WELL,CHNG,TJX,IBM,GEVO,TMHC,GOOG,DIS,INVH,FAST,LEN,VISL,VTR,ATVI,DGLY,IVZ,NOV,VXRT,AMGN,DUK,FOXA,K,DELL,MBIO,ICLK,DVAX,FIS,SAVE,WY,CGIX,MARA,MPW,TMUS,INSG,GS,DT,HBI,KHC,LKQ,NEE,QCOM,ON,ALXN,CDEV,SWN,FCX,NDRA,D,KSS,ETSY,NCLH,PSEC,X,VST,IDEX,CI,WORK,JBLU,GLW,RESN,MSFT,JWN,NVAX,BRX,VICI,LTHM,AVTR,RCL,SBUX,MA,DAL,BCRX,LYFT,CRK,NIO,NWL,KOPN,JPM,CTSH,MIK,COST,LUV,PRTS,SIRI,TSN,FISV,PLD,DHI,AMAT,ADI,CFG,EXEL,WFC,EBAY,ADMA,AMZN,PM,WU,MO,MPC,AMC,VZ,BNGO,SPG,KIM,NEM,COCP,USB,OBLN,APPS,PK,ACRS,NFLX,DRE,WKHS,AAPL,FOX,AIV,CCL,FB,JNPR,INFI,ISBC,AEP,MAXR,AXTA,ADBE,WMT,BYND,TPH,TDOC,AMRS,TELL,COTY,MU,KDP,PINS,SFM,FE,MVIS,FNB,CYRX,EA,GIS,AQB,WYNN,V,MARK,VSTM,STCN,TWTR,BX,LOW,ORGO,RIOT,RRC,HAL,TSLA,CAG,FTI,LITE,GILD,ZM,ATUS,CRM,GNW,SRNE,IPG,COMM,HMHC,LPCN,MMM,CNK,EQR,ROKU,CXW,JCI,CLDR,UBER,SPPI,AR,M,VNO,CLNE,BE,BOXL,VUZI,GRTS,SEE,MOS,KBH,CNX,AHT,PHM,IZEA,SCHW,KR,VLO,STT,NKE,EQH,MS,HL,BLNK,ETRN,EQT'

In [3]:
# !pip install xgboost